<a href="https://colab.research.google.com/github/JLDynamics/CV/blob/main/z_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Z-Image-Turbo Image Generation in Colab

This notebook contains the complete working code to generate images using Z-Image-Turbo in Google Colab.

✅ **All setup complete!** Just run the cells below to start generating images.

In [ ]:
# Step 1: Install required packages and set up Google Drive
print("📦 Installing required packages...")
!pip install git+https://github.com/huggingface/diffusers
!pip install torch accelerate transformers pillow
print("✅ Installation complete!")

# Mount Google Drive to access the model
print("💾 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

In [ ]:
# Step 2: Import libraries and check environment
import torch
from diffusers import ZImagePipeline
from IPython.display import display
import os

print("🔍 Checking environment...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU detected, using CPU (will be very slow)")

In [ ]:
# Step 3: Set up model path and copy from Google Drive
print("📁 Setting up model path...")

# Define where the model is stored in your Google Drive
# Replace this path with where you uploaded the Z-Image-Turbo folder
drive_model_path = "/content/drive/MyDrive/Z-Image-Turbo"
colab_model_path = "/content/Z-Image-Turbo"

# Check if model exists in Google Drive
if os.path.exists(drive_model_path):
    print(f"✅ Model found in Google Drive at: {drive_model_path}")

    # Copy model from Google Drive to Colab workspace
    print("📂 Copying model to Colab workspace...")
    !cp -r "{drive_model_path}" "{colab_model_path}"
    print("✅ Model copied successfully!")
else:
    print(f"❌ Model not found in Google Drive at: {drive_model_path}")
    print("Please upload the Z-Image-Turbo folder to your Google Drive first")
    print("Then update the drive_model_path variable above to match your upload location")

In [ ]:
# Step 4: Load the Z-Image-Turbo model
print("🚀 Loading Z-Image-Turbo model...")

# Clear CUDA cache and collect garbage to free up memory
import gc
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 Cleared CUDA cache and collected garbage.")

try:
    # Load with appropriate dtype based on available hardware
    dtype = torch.bfloat16 if device == "cuda" else torch.float32

    pipe = ZImagePipeline.from_pretrained(
        colab_model_path,  # Use the copied model path
        torch_dtype=dtype,
        low_cpu_mem_usage=True, # Changed to True for memory optimization
    )

    if device == "cuda":
        pipe.to("cuda")

    print("✅ Model loaded successfully!")

    # Optional: Enable Flash Attention for better performance
    try:
        pipe.transformer.set_attention_backend("flash")
        print("⚡ Flash Attention enabled")
    except:
        print("🔧 Flash Attention not available")

    # Optional: Compile model for faster inference (disabled for troubleshooting shape mismatch)
    # try:
    #     pipe.transformer.compile()
    #     print("🔨 Model compiled for faster execution")
    # except:
    #     print("🔧 Model compilation not available")

except Exception as e:
    print(f"❌ Failed to load model: {e}")
    print(f"Please ensure the model is properly copied to: {colab_model_path}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import torch
import os

# Define resolution map
resolution_map = {
    'Low': (512, 512),
    'Medium': (768, 768),
    'High': (1024, 1024),
    '1080p': (1920, 1080)
}

print("✅ Libraries imported and resolution map defined.")

In [ ]:
print("⚙️ Creating interactive interface...")

# Create widgets
prompt_widget = widgets.Textarea(
    value="A majestic lion roaring in the savanna, golden hour, highly detailed, photorealistic",
    description='Prompt:',
    layout=widgets.Layout(width='auto', height='100px')
)

resolution_selector = widgets.Dropdown(
    options=list(resolution_map.keys()),
    value='High',
    description='Resolution:'
)

num_inference_steps_widget = widgets.IntSlider(
    value=9,
    min=1,
    max=20,
    step=1,
    description='Inference Steps:',
    continuous_update=False
)

random_seed_widget = widgets.IntText(
    value=42,
    description='Random Seed:'
)

generate_button = widgets.Button(
    description='Generate Image',
    button_style='success',
    icon='magic'
)

output_area = widgets.Output()

def on_generate_button_clicked(b):
    with output_area:
        clear_output()
        current_prompt = prompt_widget.value
        selected_resolution_key = resolution_selector.value
        current_num_inference_steps = num_inference_steps_widget.value
        current_random_seed = random_seed_widget.value

        width, height = resolution_map[selected_resolution_key]

        print("Generating image...")
        print(f"Prompt: {current_prompt[:100]}...")
        print(f"Resolution: {width}x{height}")
        print(f"Inference Steps: {current_num_inference_steps}")
        print(f"Random Seed: {current_random_seed}")

        if selected_resolution_key == '1080p':
            print("⚠️ Warning: High resolution (1080p) might be slow and require significant VRAM.")

        try:
            generator = torch.Generator(device).manual_seed(current_random_seed)
            image = pipe(
                prompt=current_prompt,
                height=height,
                width=width,
                num_inference_steps=current_num_inference_steps,
                guidance_scale=0.0,
                generator=generator,
            ).images[0]

            display(image)
            output_filename = f"generated_image_{width}x{height}_seed{current_random_seed}.png"
            image.save(output_filename)
            print(f"✅ Image saved as: {output_filename}")
        except Exception as e:
            print(f"❌ Image generation failed: {e}")

generate_button.on_click(on_generate_button_clicked)

display(
    widgets.VBox([
        prompt_widget,
        resolution_selector,
        num_inference_steps_widget,
        random_seed_widget,
        generate_button,
        output_area
    ])
)
print("✅ Interactive interface created.")